In [4]:
import pickle

def load_emb(fn_prefix):
    with open(f"{fn_prefix}_train_emb.pkl", "rb") as f:
        emb_dict = pickle.load(f)
        train_ids, train_emb = emb_dict['prot_id'], emb_dict['embedding']
    with open(f"{fn_prefix}_test_emb.pkl", "rb") as f:
        emb_dict = pickle.load(f)
        test_ids, test_emb = emb_dict['prot_id'], emb_dict['embedding']
    return {"train_ids": train_ids, "train_emb": train_emb, "test_ids": test_ids, "test_emb": test_emb}

rostlab_emb = load_emb("emb/rostlab")

In [9]:
import numpy as np
import json, torch
from go_metric.data_utils import *
train_path = "/home/andrew/go_metric/data/go_bench"
train_dataset = BertSeqDataset.from_pickle(f"{train_path}/train.pkl")
val_dataset = BertSeqDataset.from_pickle(f"{train_path}/val.pkl")
test_dataset = BertSeqDataset.from_memory("data/go_bench/testing_molecular_function_annotations.tsv", 
                                            "data/go_bench/molecular_function_terms.json", 
                                            "data/uniprot_reviewed.fasta")
test_dataset.to_pickle(f"{train_path}/test.pkl")

In [ ]:
import faiss
def embedding_knn(db_emb, q_emb, db_labels, k=100):
    n_train, dim = db_emb.shape
    index = faiss.IndexFlatL2(dim)
    index.add(db_emb)
    distances, neighbors = index.search(q_emb, k)
    # matches = knn(db_emb, q_emb, k).cpu().numpy() # 2 x N*K
    # distances, neighbors = distances.numpy(), neighbors.numpy()
    q_labels = np.zeros((q_emb.shape[0], db_labels.shape[1]), dtype=float)
    for i in range(q_emb.shape[0]):
        db_res = neighbors[i]
        db_avg = db_labels[db_res].sum(axis=0) / k
        q_labels[i] = db_avg
    return q_labels
